> Это пример решения задачи с использованием Keras. Вы можете использовать этот кернер для дальнейших исследований и экспериментов.
# Классификация изображений

### Основная идея этого решения: взять предобученую на ImageNet сеть Xception и дообучить под нашу задачу. 
По ходу решения мы будем давать вам рекомендации, которые помогут улучшить качество модели. 


Удачи и Поехали!

In [1]:
!nvidia-smi

In [2]:
#Добавим более продвинутые библиотеки аугментации изображений (наприме, albumentations). Для них есть специальные «обёртки» под Keras, например ImageDataAugmentor
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor
# Устанавливаем модель efficientnet
!pip install efficientnet

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os
import albumentations


import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import *

from sklearn.model_selection import train_test_split, StratifiedKFold

from ImageDataAugmentor.image_data_augmentor import *
from tensorflow.keras.applications import EfficientNetB7
import efficientnet.tfkeras as efn
from tensorflow.keras.models import Model, Sequential
import tensorflow.keras.layers as layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

import PIL
from PIL import ImageOps, ImageFilter
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

print(os.listdir("../input"))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

**Работаем с Tensorflow v2**

In [4]:
!pip freeze > requirements.txt

# Основные настройки

In [5]:
# В setup выносим основные настройки: так удобнее их перебирать в дальнейшем.

RANDOM_SEED = 42
EPOCHS               = 8  # эпох на обучение
BATCH_SIZE           = 32 # уменьшаем batch если сеть большая, иначе не влезет в память на GPU
LR                   = 1e-3
VAL_SPLIT            = 0.2 # сколько данных выделяем на тест = 20%

CLASS_NUM            = 10  # количество классов в нашей задаче
IMG_SIZE             = 224 # какого размера подаем изображения в сеть
IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

DATA_PATH = '../input/'
PATH = "../working/car/" # рабочая директория

# EDA / Анализ данных

In [6]:
train_df = pd.read_csv(DATA_PATH+"train.csv")
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")
train_df.head()

In [7]:
train_df.info()

In [8]:
train_df.Category.value_counts()
# распределение классов достаточно равномерное - это хорошо

In [9]:
print('Распаковываем картинки')
# Will unzip the files so that you can see them..
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile("../input/"+data_zip,"r") as z:
        z.extractall(PATH)
        
print(os.listdir(PATH))

In [10]:
print('Пример картинок (random sample)')
plt.figure(figsize=(12,8))

random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(PATH+f'train/{random_image_cat[index]}/{path}')
    plt.subplot(3,3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(random_image_cat[index]))
    plt.axis('off')
plt.show()

Посмотрим на примеры картинок и их размеры чтоб понимать как их лучше обработать и сжимать.

In [11]:
image = PIL.Image.open(PATH+'/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

# Подготовка данных

### Аугментация данных

In [12]:
# Настройка условий для аугментации
AUGMENTATIONS = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.Rotate(limit=30, 
                          interpolation=1, 
                          border_mode=4, 
                          value=None, 
                          mask_value=None, 
                          always_apply=False, 
                          p=0.5),
    albumentations.OneOf([
        albumentations.CenterCrop(height=224, 
                                  width=200),
        albumentations.CenterCrop(height=200, 
                                  width=224),
    ],p=0.5),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.3, 
                                                contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(brightness_limit=0.1, 
                                                contrast_limit=0.1)
    ],p=0.5),
    albumentations.GaussianBlur(p=0.05),
    albumentations.HueSaturationValue(p=0.5),
    albumentations.RGBShift(p=0.5),
    albumentations.FancyPCA(alpha=0.1, 
                            always_apply=False, 
                            p=0.5),
    albumentations.Resize(IMG_SIZE, IMG_SIZE)
])

In [13]:
train_datagen = ImageDataAugmentor(
    rescale=1./255,
    augment=AUGMENTATIONS,
    validation_split=VAL_SPLIT
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

### Генерация данных

In [14]:
# Завернем наши данные в генератор:

train_generator = train_datagen.flow_from_directory(
    PATH+'train/',      # директория где расположены папки с картинками 
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') # set as training data

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') # set as validation data

In [15]:
# посмотрим результаты
train_generator.show_data()

# Построение модели

**При проведении анализа вариантов предобученных моделей остановились на EfficientNetB7 т.к. Xception и InceptionV3 менее эффективны**

In [16]:
base_model = efn.EfficientNetB7(
    weights='imagenet', include_top=False, input_shape=input_shape)

In [17]:
base_model.summary()

**Давайте зафиксируем веса в базовой модели, чтобы мы не могли стереть функции, полученные в ImageNet в начале обучения.**

In [18]:
base_model.trainable = False

In [19]:
# Устанавливаем новую "голову" (head)
model = Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.25))
model.add(layers.Dense(CLASS_NUM, activation='softmax'))

In [20]:
model.summary()

In [21]:
# проверка статуса слоев
for layer in model.layers:
    print(layer, layer.trainable)

## Обучение модели

In [22]:
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

Добавим ModelCheckpoint чтоб сохранять прогресс обучения модели и можно было потом подгрузить и дообучить модель.

In [23]:
checkpoint = ModelCheckpoint('best_model.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')
earlystop = EarlyStopping(monitor='accuracy', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=2, min_lr=0.0000001, verbose=1, mode='auto')
callbacks_list = [checkpoint, earlystop, reduce_lr]

Обучаем:

In [24]:
history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples//train_generator.batch_size,
        validation_data=test_generator, 
        validation_steps=test_generator.samples//test_generator.batch_size,
        epochs=EPOCHS,
        callbacks=callbacks_list
)

In [25]:
# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
model.save('../working/model_1.hdf5')
model.load_weights('best_model.hdf5')

In [26]:
scores = model.evaluate(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Точность нашей модели составила 70%. 
Посмотрим графики обучения:

In [27]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()


**Обучение половины весов**

In [28]:
# Количеcтво слоев в базовой модели
print("Number of layers in the base model: ", len(base_model.layers))

In [29]:
base_model.trainable = True

fine_tune_at = len(base_model.layers)//2

# остальные слои заморозим 
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

# проверим количество слоев
print("Number of layers in the base model: ", len(base_model.trainable_variables))


In [30]:
#Создание модели
model.compile(loss="categorical_crossentropy", 
              optimizer=optimizers.Adam(lr=LR), 
              metrics=["accuracy"])

In [31]:
#Обучение
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data = test_generator, 
    validation_steps = test_generator.samples//test_generator.batch_size,
    epochs = EPOCHS,
    callbacks = callbacks_list
    )

In [32]:
#сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
model.save('../working/model_2.hdf5')
model.load_weights('best_model.hdf5')

In [33]:
scores = model.evaluate(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Точность нашей модели составила 93%. 
Посмотрим графики обучения:

In [34]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

**Разморозим всю сеть EfficientNetB7**

In [38]:
base_model.trainable = True

In [39]:
BATCH_SIZE = 8 # уменьшим BATCH_SIZE
LR=0.000001 # изменим learning_rate

train_generator = train_datagen.flow_from_directory(
    PATH+'train/',      
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') 

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') 

In [40]:
#Создание модели

model.compile(loss="categorical_crossentropy", 
              optimizer=optimizers.Adam(lr=LR), 
              metrics=["accuracy"])

In [41]:
# Обучание

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data = test_generator, 
    validation_steps = test_generator.samples//test_generator.batch_size,
    epochs = EPOCHS,
    callbacks = callbacks_list
    )

In [52]:
#сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
model.save('../working/model_3.hdf5')
model.load_weights('best_model.hdf5')

In [53]:
scores = model.evaluate(test_generator, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Точность нашей модели составила 95%. 
Посмотрим графики обучения:

In [54]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

# Предсказание на тестовых данных

In [58]:
test_sub_generator = test_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE)

In [59]:
test_sub_generator.samples

In [60]:
test_sub_generator.reset()
predictions = model.predict_generator(test_sub_generator, steps=len(test_sub_generator), verbose=1) 
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [61]:
filenames_with_dir=test_sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission.csv', index=False)
print('Save submit')

# **Test Time Augmentation (TTA)**

Дополним тестовые изображения и сделаем несколько прогнозов для одного и того же изображения, дополненного разными способами. Затем мы возьмем среднее значение этих прогнозов и, таким образом, получим окончательный прогноз

In [62]:
model.load_weights('best_model.hdf5')

In [63]:
AUGMENTATIONS = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.Rotate(limit=30, 
                          interpolation=1, 
                          border_mode=4, 
                          value=None, 
                          mask_value=None, 
                          always_apply=False, 
                          p=0.5),
    albumentations.OneOf([
        albumentations.CenterCrop(height=224, 
                                  width=200),
        albumentations.CenterCrop(height=200, 
                                  width=224),
    ],p=0.5),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.3, 
                                                contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(brightness_limit=0.1, 
                                                contrast_limit=0.1)
    ],p=0.5),
    albumentations.GaussianBlur(p=0.05),
    albumentations.HueSaturationValue(p=0.5),
    albumentations.RGBShift(p=0.5),
    albumentations.FancyPCA(alpha=0.1, 
                            always_apply=False, 
                            p=0.5),
    albumentations.Resize(IMG_SIZE, IMG_SIZE)
])

In [64]:
test_datagen = ImageDataAugmentor( 
    rescale=1./255,
    augment=AUGMENTATIONS,
    validation_split=VAL_SPLIT
)

In [65]:
test_sub_generator = test_datagen.flow_from_dataframe(
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=False,
    seed=RANDOM_SEED
)

In [66]:
tta_steps = 10 
predictions = []

for i in range(tta_steps):
    preds = model.predict_generator(test_sub_generator, 
                                    steps=len(test_sub_generator), 
                                    verbose=1) 
    predictions.append(preds)

pred = np.mean(predictions, axis=0)
predictions = np.argmax(pred, axis=-1)
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items())
predictions = [label_map[k] for k in predictions]

In [67]:
filenames_with_dir=test_sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission_TTA.csv', index=False)
print('Save submit')

In [71]:
scores = model.evaluate(test_generator,verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
submission.head()

In [ ]:
# Clean PATH
import shutil
shutil.rmtree(PATH)

**Выводы**

В работе примнил методы:

transfer learning и fine-tuning

настройка LR, optimizer

подобраны переменные (размер картинки, батч и т.д.)

SOTA архитектура сетей - EfficientNetB7

добавлена Batch Normalization и изменена архитектура “головы”

применены дополнительные функции callback Keras https://keras.io/callbacks/

TTA (Test Time Augmentation)